# Microwave source interface

This instrument is an interface for a real microwave source or can be used to simulate one. The idea being that the underlying instrument can be swapped out but the API remains fixed so the rest of the code in use stays the same.

A simulated microwave source initialises with defaults. A real microwave source will also initialise with parameter values as None (but if you call get it will update them)<br><br>
NB: status, IQ_state, dual_output and pulsemod_state can be set with any (reasonable) bool-like argument and the 'raw value' set on the simulated instrument will be 1 or 0 but they will always return booleans as per the on_off_val_mapping in use. On a real instrument IQ_state and dual_output_state can only be set if the instrument has the IQ option (and is not CW).

In [1]:
from qcodes import Instrument
from qdev_wrappers.station_configurator import StationConfigurator
from qdev_wrappers.customised_instruments.interfaces.microwave_source_interface import SimulatedMicrowaveSourceInterface, SGS100AExternalMicrowaveSourceInterface, SGS100AMicrowaveSourceInterface

/Users/natalie/Documents/PhD/Qdev/QdevWrappers/qdev_wrappers/logger.py:16: UserWarning: The logger.py of qdev-wrappers is deprecated and will be removed soon. Please use the logger of QCoDeS instead.
  warnings.warn('The logger.py of qdev-wrappers is deprecated and will be '
/Users/natalie/Documents/PhD/Qdev/QdevWrappers/qdev_wrappers/station_configurator.py:21: UserWarning: ruamel yaml not found station configurator is falling back to pyyaml. It's highly recommended to install ruamel.yaml. This fixes issues with scientific notation and duplicate instruments in the YAML file
  warnings.warn("ruamel yaml not found station configurator is falling back to pyyaml. "


In [2]:
real_instrument_present = False

In [3]:
def print_params(instr):
    print(instr.name.upper())
    for k, v in instr.parameters.items():
        if k != 'IDN':
            print(k, ': ', v())
    print('\n')

In [9]:
try:
    Instrument.remove_instance(source_if)
    Instrument.remove_instance(instrument)
except NameError:
    pass
if real_instrument_present:
    sc = StationConfigurator()
    instrument = sc.load_instrument('microwave_source')
    source_if = SGS100AMicrowaveSourceInterface('real_source', instrument)
else:
    source_if = SimulatedMicrowaveSourceInterface('simulated_source')

print_params(source_if)

SIMULATED_SOURCE
frequency :  7000000000.0
power :  -10
status :  0
pulsemod_state :  0
IQ_state :  0
dual_output_state :  0




In [5]:
source_if.frequency(8e9)
source_if.power(-15)
source_if.status(1)
source_if.IQ_state(0)
source_if.pulsemod_state(1)
print_params(source_if)

SIMULATED_SOURCE
frequency :  8000000000.0
power :  -15
status :  1
pulsemod_state :  1
IQ_state :  0
dual_output_state :  0




Since some sources do not have the IQ option in hardware this can be entered manually on initialisation and the parameter will not be settable and will be fixed to off. The same is true for the dual_output option. <br> <br>If an external mixer is present it is possible to use the interface where the IQ_state parameter will not be settable but will be fixed to on. The latter option is only available for the real instruments.

In [11]:
# choose True for the external mixer option and False for the no IQ
fixed_IQ_state = False

In [12]:
try:
    Instrument.remove_instance(cw_source_if)
except NameError:
    pass

if fixed_IQ_state:
    cw_source_if = SGS100AExternalMicrowaveSourceInterface('real_source_cw', instrument)
elif real_instrument_present:
    cw_source_if = SGS100AMicrowaveSourceInterface('real_source_cw', instrument, IQ=False)
else:
    cw_source_if = SimulatedMicrowaveSourceInterface('simulated_source_cw', IQ=False)

try:
    cw_source_if.IQ_state(1)
except RuntimeError as e:
    print(e)

('Parmeter IQ_state not settable', 'setting simulated_source_cw_IQ_state to 1')
